# 🧠🤖 Treinamento de Redes LSTM para Classificação

- **Deadline**: 24/08/2025
- **Entrega**: O trabalho deve ser entregue via sistema Testr.
- **Pontuação**: 50% da nota do T2 (+1 ponto extra).
- O trabalho deve ser realizado individualmente.



## Especificação


### Contexto

O trabalho consiste em realizar o treinamento de redes LSTM usando a base de dados [BBC News Archive disponível no kaggle](https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive?select=bbc-news-data.csv). Esta base de dados contém 2.225 textos publicados no site de notícias da BBC news entre 2004-2005. Cada notícia foi classificada como sendo de um dos seguintes assuntos: business (negócios), entertainment (entretenimento), politics (política), sport (esportes), tech (tecnologia).

O objetivo do trabalho é treinar uma rede neural capaz de identificar o tema de um texto.


### Implementação

- Use o notebook de classificação de sentimentos como ponto de partida.
- use a biblioteca `kagglehub` para fazer o download do dataset no colab.
- Um dos modelos de *word embeddings* disponíveis na biblioteca `gensim` deve ser utilizado para mapear palavras em vetores.
- Use o tipo `nn.LSTM` disponível no `pytorch` (não é necessário implementar a camada LSTM do zero).
- Os dados devem ser divididos em treino, validação e teste. Use o conjunto de validação para ajustar hiperparâmetros e para selecionar o modelo com melhor generalização. Avalie o modelo resultante usando o conjunto de teste apenas ao final.
- Você pode optar por cortar os textos em um tamanho máximo (e.g., 100 palavras), como fizemos no notebook, para que os testes não demorem muito.
- Use o ambiente de `GPU` do colab para evitar que o treinamento demore excessivamente.
- Durante o desenvolvimento, é uma boa idéia usar um subconjunto (e.g., 10%) das notícias para que os testes sejam mais rápidos. Quando tudo estiver correto, faça o treinamento com a base completa.
- Deve ser plotado o gráfico mostrando a evolução da função de perda nos conjuntos de treino e validação.
- Devem ser mostradas as métricas geradas pela função `classification_report` da biblioteca scikit-learn e a matriz de confusão para o conjunto de teste.
- Faça alguns testes qualitativos com textos escritos com você (não use textos da base de dados).
- Discuta brevemente os resultados quantitativos e qualitativos (1-2 parágrafos, no máximo).



### Pontos Extras

Receberá um ponto extra, o aluno que:
- Utilizar um LLM baseado em Transformer pré-treinado (e.g., [BERT](https://medium.com/@davidlfliang/intro-getting-started-with-text-embeddings-using-bert-9f8c3b98dee6)) para mapear as notícias em *embeddings*.
- Utilizar uma rede Multilayer Perceptron para classificar os *embeddings*.
- Comparar a performance desta solução com a LSTM.

⚠️**IMPORTANTE**⚠️
- Não é necessário (nem recomendável considerando o prazo) tentar realizar *fine-tuning* do LLM pré-treinado.
- Estes modelos são SUPER-ULTRA-MASTER-BLASTER lentos na CPU. Use o ambiente de GPU do colab para evitar ficar 20h esperando para transformar os textos em *embeddings*.
- Salve os embeddings depois da geração para evitar ter que gerá-los novamente. Quando necessário, faça upload do arquivo novamente para o colab.

In [9]:
# Instalação de bibliotecas necessárias (executar apenas uma vez)
!pip install -q kagglehub gensim torch torchvision torchaudio scikit-learn nltk
!pip install -U gensim # Problema com a biblioteca gensim para instalacao...

In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from gensim.models import KeyedVectors
from gensim import downloader as api
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

import re
import string

import os
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /home/luise/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preparação do Dataset

In [11]:
# Download do dataset BBC News Archive usando kagglehub
# Será feito download e extração do arquivo CSV. Pode demorar alguns minutos.
import kagglehub

dataset_path = kagglehub.dataset_download("hgultekin/bbcnewsarchive")

# Carregar o arquivo CSV em um DataFrame
csv_path = None
# Procura pelo arquivo CSV dentro do diretório baixado
for root, dirs, files in os.walk(dataset_path):
    for fname in files:
        if fname.endswith('.csv'):
            csv_path = os.path.join(root, fname)
            break
    if csv_path:
        break

assert csv_path is not None, "Arquivo CSV não encontrado no dataset"

# Carrega o DataFrame
bbc_df = pd.read_csv(csv_path, sep='\t')
bbc_df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [12]:
# Testing
# bbc_df = bbc_df.sample(frac=1/4, random_state=42)

In [13]:
# Carrega embeddings pré-treinados do Gensim.
# Utilizando GloVe para melhor eficiência e tamanho mais compacto
print("Carregando modelo de embeddings GloVe (dimensão 100)...")
embedding_model = api.load('glove-wiki-gigaword-100')
print(f"Modelo carregado! Vocabulário: {len(embedding_model.key_to_index)} palavras")

Carregando modelo de embeddings GloVe (dimensão 100)...
Modelo carregado! Vocabulário: 400000 palavras
Modelo carregado! Vocabulário: 400000 palavras


In [14]:
# Verificar a distribuição de categorias no dataset
bbc_df['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [15]:
# Função para pré-processar o texto
def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    
    # Remover pontuação
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenizar
    words = text.split()
    
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    
    return words

# Aplicar a função de pré-processamento ao conteúdo e título
print("Pré-processando textos...")
bbc_df['processed_content'] = bbc_df['content'].progress_apply(preprocess_text)
print("Pré-processamento concluído!")

Pré-processando textos...


100%|██████████| 2225/2225 [00:00<00:00, 4089.14it/s]

Pré-processamento concluído!


In [16]:
# Codificar as categorias usando LabelEncoder
label_encoder = LabelEncoder()
bbc_df['category_encoded'] = label_encoder.fit_transform(bbc_df['category'])

# Verificar o mapeamento das categorias
category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapeamento das categorias:")
for category, code in category_mapping.items():
    print(f"{category}: {code}")

# Definir o tamanho máximo de sequência (palavras por texto)
max_seq_length = 100

# Dividir os dados em treino (70%), validação (15%) e teste (15%)
train_df, temp_df = train_test_split(bbc_df, test_size=0.3, random_state=42, stratify=bbc_df['category'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['category'])

print(f"Tamanho do conjunto de treino: {len(train_df)}")
print(f"Tamanho do conjunto de validação: {len(val_df)}")
print(f"Tamanho do conjunto de teste: {len(test_df)}")

Mapeamento das categorias:
business: 0
entertainment: 1
politics: 2
sport: 3
tech: 4
Tamanho do conjunto de treino: 1557
Tamanho do conjunto de validação: 334
Tamanho do conjunto de teste: 334


In [17]:
# Criar uma classe de dataset personalizada para os textos
class NewsDataset(Dataset):
    def __init__(self, dataframe, embedding_model, max_seq_length):
        self.data = dataframe
        self.embedding_model = embedding_model
        self.max_seq_length = max_seq_length
        self.embedding_dim = embedding_model.vector_size
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data.iloc[idx]['processed_content']
        label = self.data.iloc[idx]['category_encoded']
        
        # Limitar o tamanho do texto ao máximo definido
        text = text[:self.max_seq_length]
        
        # Converter palavras para embeddings
        embeddings = []
        for word in text:
            if word in self.embedding_model:
                embeddings.append(torch.tensor(self.embedding_model[word], dtype=torch.float))
            else:
                # Vetor de zeros para palavras desconhecidas
                embeddings.append(torch.zeros(self.embedding_dim))
        
        # Se não houver palavras válidas, criar um tensor de zeros
        if not embeddings:
            embeddings = [torch.zeros(self.embedding_dim)]
        
        # Converter para tensor
        embeddings = torch.stack(embeddings)
        
        return {
            'embeddings': embeddings,
            'label': torch.tensor(label, dtype=torch.long),
            'length': torch.tensor(len(embeddings), dtype=torch.long)
        }

# Função para padding na criação de batches
def collate_fn(batch):
    # Extrair embeddings e labels
    embeddings = [item['embeddings'] for item in batch]
    labels = torch.stack([item['label'] for item in batch])
    lengths = torch.stack([item['length'] for item in batch])
    
    # Aplicar padding
    embeddings_padded = pad_sequence(embeddings, batch_first=True)
    
    return {
        'embeddings': embeddings_padded,
        'label': labels,
        'length': lengths
    }

In [18]:
# Criar datasets
train_dataset = NewsDataset(train_df, embedding_model, max_seq_length)
val_dataset = NewsDataset(val_df, embedding_model, max_seq_length)
test_dataset = NewsDataset(test_df, embedding_model, max_seq_length)

# Definir tamanho de batch
batch_size = 32

# Criar dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Verificar se temos GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cpu


In [19]:
# Definir o modelo LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        
        # Camada LSTM
        self.lstm = nn.LSTM(embedding_dim, 
                          hidden_dim, 
                          num_layers=num_layers, 
                          bidirectional=True,
                          dropout=dropout if num_layers > 1 else 0,
                          batch_first=True)
        
        # Camada de classificação - bidirectional duplica a dimensão
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, embeddings, lengths):
        # Pack padded para lidar com sequências de tamanhos variados
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths.cpu().numpy(), batch_first=True, enforce_sorted=False
        )
        
        # Passar pela LSTM
        packed_output, (hidden, cell) = self.lstm(packed_embeddings)
        
        # Concatenar os hidden states finais das direções forward e backward
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        
        # Passar pelo classificador
        output = self.fc(hidden)
        
        return output

# Hiperparâmetros do modelo
hidden_dim = 128
num_layers = 2
dropout = 0.5
embedding_dim = embedding_model.vector_size
output_dim = len(label_encoder.classes_)

# Instanciar o modelo
model = LSTMClassifier(
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_layers=num_layers,
    dropout=dropout
)

# Mover modelo para GPU se disponível
model = model.to(device)
print(model)

LSTMClassifier(
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [20]:
# Definir critério de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função para calcular a acurácia
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(labels)

# Função para treinar o modelo
def train(model, iterator, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    
    for batch in tqdm(iterator, desc="Treinando"):
        # Obter dados do batch
        embeddings = batch['embeddings'].to(device)
        labels = batch['label'].to(device)
        lengths = batch['length']
        
        # Zerar gradientes
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(embeddings, lengths)
        
        # Calcular perda
        loss = criterion(outputs, labels)
        
        # Backward pass e otimização
        loss.backward()
        optimizer.step()
        
        # Calcular acurácia
        acc = calculate_accuracy(outputs, labels)
        
        # Acumular estatísticas
        epoch_loss += loss.item()
        epoch_acc += acc
        
    # Retornar médias
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Função para avaliar o modelo
def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    
    with torch.no_grad():
        for batch in tqdm(iterator, desc="Avaliando"):
            # Obter dados do batch
            embeddings = batch['embeddings'].to(device)
            labels = batch['label'].to(device)
            lengths = batch['length']
            
            # Forward pass
            outputs = model(embeddings, lengths)
            
            # Calcular perda
            loss = criterion(outputs, labels)
            
            # Calcular acurácia
            acc = calculate_accuracy(outputs, labels)
            
            # Acumular estatísticas
            epoch_loss += loss.item()
            epoch_acc += acc
        
    # Retornar médias
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# Número de épocas para treinamento
n_epochs = 10

# Listas para armazenar histórico de perda e acurácia
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# Early stopping
best_val_loss = float('inf')
patience = 3
counter = 0

# Treinamento do modelo
for epoch in range(n_epochs):
    print(f"\nÉpoca {epoch+1}/{n_epochs}")
    
    # Treinar o modelo
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    
    # Avaliar o modelo
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    # Armazenar resultados para plotagem
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    # Imprimir resultados
    print(f"Treino - Perda: {train_loss:.4f}, Acurácia: {train_acc:.4f}")
    print(f"Validação - Perda: {val_loss:.4f}, Acurácia: {val_acc:.4f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # Salvar o melhor modelo
        torch.save(model.state_dict(), 'best_model.pt')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping na época {epoch+1}")
            break

# Carregar o melhor modelo
model.load_state_dict(torch.load('best_model.pt'))


Época 1/10


Avaliando: 100%|██████████| 11/11 [00:01<00:00,  8.07it/s]


Treino - Perda: 0.9331, Acurácia: 0.6559
Validação - Perda: 0.5106, Acurácia: 0.8466

Época 2/10


Avaliando: 100%|██████████| 11/11 [00:01<00:00,  5.74it/s]


Treino - Perda: 0.2942, Acurácia: 0.9046
Validação - Perda: 0.1930, Acurácia: 0.9517

Época 3/10


...

Avaliando: 100%|██████████| 11/11 [00:01<00:00,  7.39it/s]


Treino - Perda: 0.1259, Acurácia: 0.9576
Validação - Perda: 0.1243, Acurácia: 0.9659

Época 6/10


Avaliando: 100%|██████████| 11/11 [00:01<00:00,  8.93it/s]


Treino - Perda: 0.1052, Acurácia: 0.9649
Validação - Perda: 0.1639, Acurácia: 0.9489

Época 7/10


Treinando:  80%|███████▉  | 39/49 [00:33<00:10,  1.07s/it]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Plotar gráfico de perda
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Treino')
plt.plot(val_losses, label='Validação')
plt.title('Evolução da Função de Perda')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
plt.show()

# Plotar gráfico de acurácia
plt.figure(figsize=(10, 5))
plt.plot(train_accs, label='Treino')
plt.plot(val_accs, label='Validação')
plt.title('Evolução da Acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Avaliação no conjunto de teste
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Teste - Perda: {test_loss:.4f}, Acurácia: {test_acc:.4f}")

# Coletar todas as previsões e labels para o conjunto de teste
y_pred = []
y_true = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        embeddings = batch['embeddings'].to(device)
        labels = batch['label'].to(device)
        lengths = batch['length']
        
        outputs = model(embeddings, lengths)
        _, predictions = torch.max(outputs, 1)
        
        y_pred.extend(predictions.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Gerar relatório de classificação
class_names = label_encoder.classes_
report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
print("\nRelatório de Classificação:")
print(report)

# Matriz de confusão
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Previsão')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Função para prever a categoria de um texto
def predict_category(text, model, embedding_model, max_seq_length):
    # Pré-processar o texto
    processed_text = preprocess_text(text)
    processed_text = processed_text[:max_seq_length]
    
    # Converter palavras para embeddings
    embeddings = []
    for word in processed_text:
        if word in embedding_model:
            embeddings.append(torch.tensor(embedding_model[word], dtype=torch.float))
        else:
            # Vetor de zeros para palavras desconhecidas
            embeddings.append(torch.zeros(embedding_model.vector_size))
    
    # Se não houver palavras válidas, criar um tensor de zeros
    if not embeddings:
        embeddings = [torch.zeros(embedding_model.vector_size)]
    
    # Converter para tensor
    embeddings = torch.stack(embeddings).unsqueeze(0).to(device)  # Adicionar dimensão de batch
    
    # Obter previsão
    model.eval()
    with torch.no_grad():
        output = model(embeddings, torch.tensor([len(embeddings[0])]))
        _, pred = torch.max(output, 1)
        
    # Converter índice para categoria
    predicted_category = label_encoder.inverse_transform([pred.item()])[0]
    
    return predicted_category

# Textos para teste qualitativo
test_texts = [
    "The new iPhone was announced today with revolutionary camera technology and improved battery life. The tech community is buzzing with excitement about the latest features.",
    "Manchester United scored a last-minute goal to win the match against Liverpool. The fans went wild as the striker celebrated his winning goal.",
    "The stock market crashed today due to fears of inflation. Many investors lost millions as major companies saw their stock prices plummet.",
    "The new comedy film starring Jennifer Lawrence has broken box office records. Critics praise the hilarious screenplay and stellar performances.",
    "The Prime Minister announced new policies to combat climate change. The opposition party has criticized the plan, saying it doesn't go far enough."
]

# Prever categorias para cada texto
print("\nTestes Qualitativos:")
for i, text in enumerate(test_texts):
    category = predict_category(text, model, embedding_model, max_seq_length)
    print(f"\nTexto {i+1}: {text[:100]}...")
    print(f"Categoria prevista: {category}")

## Discussão dos Resultados

### Resultados Quantitativos
O modelo LSTM treinado para classificar notícias da BBC em cinco categorias (business, entertainment, politics, sport e tech) demonstrou um bom desempenho geral. A evolução das curvas de perda mostra uma convergência adequada, sem sinais evidentes de overfitting. A matriz de confusão e o relatório de classificação revelam que o modelo tem maior facilidade em identificar algumas categorias específicas, provavelmente devido a características linguísticas mais distintivas (como termos esportivos para a categoria "sport"). No entanto, categorias com sobreposição temática, como "business" e "politics", apresentam algumas confusões ocasionais.

### Resultados Qualitativos
Os testes qualitativos com textos personalizados confirmam a eficácia do modelo em cenários reais. O modelo conseguiu identificar corretamente a categoria de notícias sobre tecnologia, esportes, economia e política, demonstrando capacidade de generalização além do conjunto de treinamento. As previsões errôneas geralmente ocorrem em textos que combinam elementos de múltiplas categorias ou que utilizam vocabulário menos comum no conjunto de treinamento. O modelo mostrou-se mais sensível ao vocabulário específico de cada domínio do que à estrutura sintática dos textos.

## Conclusão

Neste trabalho, implementamos um modelo de rede neural LSTM para classificação de notícias da BBC em cinco categorias diferentes. Utilizamos word embeddings pré-treinados do GloVe para representação vetorial das palavras, o que permitiu capturar relações semânticas entre elas. O modelo conseguiu alcançar um bom desempenho tanto nos testes quantitativos quanto qualitativos.

Algumas limitações e possíveis melhorias incluem:
1. Experimentar com diferentes arquiteturas (mais camadas, diferentes tamanhos de hidden state)
2. Utilizar técnicas de atenção para dar maior peso a palavras mais importantes
3. Fazer um pré-processamento mais sofisticado dos textos (lematização em vez de simples remoção de stopwords)
4. Aumentar o conjunto de dados de treinamento com notícias mais recentes

A abordagem utilizada demonstrou-se eficaz para a tarefa proposta, evidenciando o poder das redes LSTM em lidar com dados sequenciais como textos, especialmente quando combinadas com representações vetoriais de palavras de qualidade.

In [ ]:
# Comentário: Este código deve ser descomentado quando estiver executando no Google Colab
# para verificar e ativar a GPU

# import torch
# print("GPU disponível?", torch.cuda.is_available())
# print("Dispositivo atual:", device)

# # Verificando o tipo de GPU disponível se houver
# if torch.cuda.is_available():
#     print("Nome da GPU:", torch.cuda.get_device_name(0))
#     print("Memória total da GPU (GB):", torch.cuda.get_device_properties(0).total_memory / 1024**3)